In [25]:
#mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
#inputing our data files through drive

import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/ensemble_model_with_weights_3.hdf5
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-101.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-113.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-133.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-138.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-135.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-132.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-134.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-125.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-126.png
../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/crack-o-130.p

In [27]:
# config root dir

class ROOTDIR:
    train_images_dir = "../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/images/"
    train_masks_dir = "../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/"

In [28]:
import tensorflow as tf  # good



import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras
import pandas as pd
from datetime import datetime

from keras.utils import normalize
#from keras.metrics import MeanIoU


#Resizing images, if needed
SIZE_X = 256
SIZE_Y = 256
n_classes=2 #Number of classes for segmentation

#Capture training image info as a list
train_images = []

image_paths = sorted(glob.glob("/content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/images/*.jpg"))

for img_path in image_paths:

        img = cv2.imread(img_path, 1)
       # img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)

#Convert list to array for machine learning processing
train_images = np.array(train_images)

#Capture mask/label info as a list
# Capture mask/label info as a list
train_masks = []

mask_paths = sorted(glob.glob("/content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/masks/*.png"))
for mask_path in mask_paths:
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load the mask as grayscale

    # Define a threshold value
    threshold_value = 0.5  # You may need to adjust this threshold value

    # Apply binary thresholding to convert the mask into binary
    binary_mask = np.where(mask > threshold_value, 1, 0)
    # After thresholding and converting to binary, expand the dimensions
    binary_mask = binary_mask[:, :, np.newaxis]


    # Append the binary mask to the list
    train_masks.append(binary_mask)

# Convert list to array for machine learning processing
train_masks = np.array(train_masks)


ModuleNotFoundError: No module named 'segmentation_models'

In [ ]:
 num_samples_to_check = 1  # Choose an appropriate number of samples to check

for i in range(num_samples_to_check):
    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.imshow(train_images[i])
    plt.title('Image')

    plt.subplot(1, 2, 2)
    plt.imshow(train_masks[i], cmap='gray')
    plt.title('Mask')

    plt.show()


print("Image Shape:", train_images.shape)  # Check the shape of the image array
print("Image Data Type:", train_images.dtype)  # Check the data type (should be uint8 for images)

print("Mask Shape:", train_masks.shape)  # Check the shape of the mask array
print("Mask Data Type:", train_masks.dtype)  # Check the data type (should be uint8 for masks)

print("Image Min Value:", np.min(train_images))
print("Image Max Value:", np.max(train_images))
print("Mask Min Value:", np.min(train_masks))
print("Mask Max Value:", np.max(train_masks))


num_images = len(train_images)
num_masks = len(train_masks)

print("Number of Training Images:", num_images)
print("Number of Training Masks:", num_masks)



In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing subsets (20% for testing)

X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks, test_size=0.20, random_state=0) # good




X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled

In [ ]:
print("Training images shape:", X_train.shape)
print("Training masks shape:", y_train.shape)

print("Testing images shape:", X_test.shape)
print("Testing masks shape:", y_test.shape)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a function to display an image and its corresponding mask
def visualize_sample(image, mask, title_image, title_mask):
    plt.figure(figsize=(8, 3))

    # Clip image values to [0, 255]
    image = np.clip(image, 0, 255).astype(np.uint8)

    # Display the image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(title_image)

    # Display the mask
    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap='gray')
    plt.title(title_mask)

    plt.show()

# Visualize the first 2 samples in the training dataset
for i in range(2):
    visualize_sample(X_train[i], y_train[i], f"Training Image Sample {i}", f"Training Mask Sample {i}")

# Visualize the first 2 samples in the testing dataset
for i in range(2):
    visualize_sample(X_test[i], y_test[i], f"Testing Image Sample {i}", f"Testing Mask Sample {i}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_samples(images, masks, titles_images, titles_masks):
    num_samples = len(images)

    fig, axs = plt.subplots(2, num_samples, figsize=(15, 6))

    for i in range(num_samples):
        image = np.clip(images[i], 0, 255).astype(np.uint8)

        axs[0, i].imshow(image)
        axs[0, i].set_title(titles_images[i])
        axs[0, i].axis('off')

        axs[1, i].imshow(masks[i], cmap='gray')
        axs[1, i].set_title(titles_masks[i])
        axs[1, i].axis('off')

    plt.tight_layout()
    plt.show()

# Define the samples to visualize (replace with your samples)
images_to_visualize = [X_train[0], X_train[1], X_train[2], X_train[3], X_train[4]]
masks_to_visualize = [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]]
titles_images = ["Training Image Sample 0", "Training Image Sample 1", "Training Image Sample 2",
                 "Training Image Sample 3", "Training Image Sample 4"]
titles_masks = ["Training Mask Sample 0", "Training Mask Sample 1", "Training Mask Sample 2",
                "Training Mask Sample 3", "Training Mask Sample 4"]

# Display the defined samples
visualize_samples(images_to_visualize, masks_to_visualize, titles_images, titles_masks)


In [ ]:
n_classes = 2  # Binary segmentation

# Use 'sigmoid' activation for binary segmentation
activation = 'sigmoid'

LR = 0.0001
optim = keras.optimizers.Adam(LR)
# Define class weights for each class
#class_weights = np.array([0.75, 0.25])  # Adjust weights as needed

dice_loss = sm.losses.DiceLoss()
binary_focal_loss = sm.losses.BinaryFocalLoss()


total_loss = dice_loss + binary_focal_loss

# Define evaluation metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]


In [ ]:
print("Data type of X_train:", X_train.dtype)
print("Data type of X_test:", X_test.dtype)
print("Data type of y_train:", y_train.dtype)
print("Data type of y_test:", y_test.dtype)


In [ ]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

optim = keras.optimizers.Adam(LR)

# preprocess input
X_train1 = preprocess_input1(X_train) #images
X_test1 = preprocess_input1(X_test)   #images
#####################################################################
###Model 1
#Using same backbone for both models

# define model (Change to unet or Linknet based on the need )
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=1, activation=activation)

# compile keras model with defined optimozer, loss and metrics




model1.compile(optim, total_loss, metrics=metrics)

#model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

print(model1.summary())

start1 = datetime.now()

history1 = model1.fit(X_train1,
                      y_train,
                      batch_size=8,
                      epochs=2,
                      verbose=1,
                      validation_data=(X_test1, y_test))

stop1 = datetime.now()

#Execution time of the model
execution_time_unet = stop1-start1
print("Unet execution time is: ", execution_time_unet)


model1.save('unet_res34_backbone_8epochs.hdf5')

# convert the history.history dict to a pandas DataFrame:
hist1_df = pd.DataFrame(history1.history)
hist1_csv_file = 'history_unet.csv'
with open(hist1_csv_file, mode='w') as f:
    hist1_df.to_csv(f)

In [ ]:

optim = keras.optimizers.Adam(LR)

model2 = sm.Linknet(BACKBONE1, encoder_weights='imagenet', classes=1, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model2.compile(optim, total_loss, metrics=metrics)

print(model2.summary())

start2 = datetime.now()

history2=model2.fit(X_train1,
          y_train,
          batch_size=8,
          epochs=2,
          verbose=1,
          validation_data=(X_test1, y_test))

stop2 = datetime.now()

#Execution time of the model
execution_time_linknet = stop2-start2
print("Linknet execution time is: ", execution_time_linknet)

model2.save('linknet_res34_backbone_8epochs.hdf5')

# convert the history.history dict to a pandas DataFrame:
hist2_df = pd.DataFrame(history2.history)
hist2_csv_file = 'history_linknet.csv'
with open(hist2_csv_file, mode='w') as f:
    hist2_df.to_csv(f)

In [ ]:

optim = keras.optimizers.Adam(LR)

model3 = sm.FPN(BACKBONE1, encoder_weights='imagenet', classes=1, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model3.compile(optim, total_loss, metrics=metrics)

print(model3.summary())

start3 = datetime.now()

history3=model3.fit(X_train1,
          y_train,
          batch_size=8,
          epochs=2,
          verbose=1,
          validation_data=(X_test1, y_test))

stop3 = datetime.now()

#Execution time of the model
execution_time_FPN = stop3-start3
print("FPN execution time is: ", execution_time_FPN)

model3.save('FPN_res34_backbone_8epochs.hdf5')

# convert the history.history dict to a pandas DataFrame:
hist3_df = pd.DataFrame(history3.history)
hist3_csv_file = 'history_FPN.csv'
with open(hist3_csv_file, mode='w') as f:
    hist3_df.to_csv(f)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, accuracy_score

# ... (your previous code for training and history1)
threshold= 0.5
# Extract the predictions for the training and validation sets
y_train_pred = model1.predict(X_train1)
y_val_pred = model1.predict(X_test1)

# Threshold the predictions to obtain binary results
y_train_pred_binary = (y_train_pred > threshold).astype(np.uint8)
y_val_pred_binary = (y_val_pred > threshold).astype(np.uint8)

# Compute precision and accuracy for training set
precision_train = precision_score(y_train.flatten(), y_train_pred_binary.flatten())
accuracy_train = accuracy_score(y_train.flatten(), y_train_pred_binary.flatten())

# Compute precision and accuracy for validation set
precision_val = precision_score(y_test.flatten(), y_val_pred_binary.flatten())
accuracy_val = accuracy_score(y_test.flatten(), y_val_pred_binary.flatten())

# Print precision and accuracy for both training and validation sets
print("Training Precision:", precision_train)
print("Training Accuracy:", accuracy_train)

print("Validation Precision:", precision_val)
print("Validation Accuracy:", accuracy_val)


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, accuracy_score

# ... (your previous code for training and history1)
threshold= 0.5
# Extract the predictions for the training and validation sets
y_train_pred = model2.predict(X_train1)
y_val_pred = model2.predict(X_test1)

# Threshold the predictions to obtain binary results
y_train_pred_binary = (y_train_pred > threshold).astype(np.uint8)
y_val_pred_binary = (y_val_pred > threshold).astype(np.uint8)

# Compute precision and accuracy for training set
precision_train = precision_score(y_train.flatten(), y_train_pred_binary.flatten())
accuracy_train = accuracy_score(y_train.flatten(), y_train_pred_binary.flatten())

# Compute precision and accuracy for validation set
precision_val = precision_score(y_test.flatten(), y_val_pred_binary.flatten())
accuracy_val = accuracy_score(y_test.flatten(), y_val_pred_binary.flatten())

# Print precision and accuracy for both training and validation sets
print("Training Precision:", precision_train)
print("Training Accuracy:", accuracy_train)

print("Validation Precision:", precision_val)
print("Validation Accuracy:", accuracy_val)


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, accuracy_score

# ... (your previous code for training and history1)
threshold= 0.5
# Extract the predictions for the training and validation sets
y_train_pred = model3.predict(X_train1)
y_val_pred = model3.predict(X_test1)

# Threshold the predictions to obtain binary results
y_train_pred_binary = (y_train_pred > threshold).astype(np.uint8)
y_val_pred_binary = (y_val_pred > threshold).astype(np.uint8)

# Compute precision and accuracy for training set
precision_train = precision_score(y_train.flatten(), y_train_pred_binary.flatten())
accuracy_train = accuracy_score(y_train.flatten(), y_train_pred_binary.flatten())

# Compute precision and accuracy for validation set
precision_val = precision_score(y_test.flatten(), y_val_pred_binary.flatten())
accuracy_val = accuracy_score(y_test.flatten(), y_val_pred_binary.flatten())

# Print precision and accuracy for both training and validation sets
print("Training Precision:", precision_train)
print("Training Accuracy:", accuracy_train)

print("Validation Precision:", precision_val)
print("Validation Accuracy:", accuracy_val)


In [ ]:
from keras.models import load_model
import numpy as np
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt

# Load your trained models
model_unet = load_model('/content/unet_res34_backbone_8epochs.hdf5', compile=False)
model_linknet = load_model('/content/linknet_res34_backbone_8epochs.hdf5', compile=False)
model_fpn = load_model('/content/FPN_res34_backbone_8epochs.hdf5', compile=False)


# Predict on test data
y_pred_unet = model_unet.predict(X_test1)
y_pred_linknet = model_linknet.predict(X_test1)
y_pred_fpn = model_fpn.predict(X_test1)


threshold = 0.5



In [ ]:
import random
from sklearn.metrics import jaccard_score

# Predict on test data
y_pred_unet_binary = (y_pred_unet > threshold).astype(np.uint8)
y_pred_linknet_binary = (y_pred_linknet > threshold).astype(np.uint8)
y_pred_fpn_binary = (y_pred_fpn > threshold).astype(np.uint8)

iou_scores_unet = []
iou_scores_linknet = []
iou_scores_fpn = []

# Compute IoU for each sample in the test set
for i in range(len(X_test1)):
    iou_unet = jaccard_score(y_test[i, :, :, 0].flatten(), y_pred_unet_binary[i, :, :, 0].flatten(), average=None)
    iou_linknet = jaccard_score(y_test[i, :, :, 0].flatten(), y_pred_linknet_binary[i, :, :, 0].flatten(), average=None)
    iou_fpn = jaccard_score(y_test[i, :, :, 0].flatten(), y_pred_fpn_binary[i, :, :, 0].flatten(), average=None)
    iou_scores_unet.append(iou_unet)
    iou_scores_linknet.append(iou_linknet)
    iou_scores_fpn.append(iou_fpn)

mean_iou_unet = np.mean(iou_scores_unet)
mean_iou_linknet = np.mean(iou_scores_linknet)
mean_iou_fpn = np.mean(iou_scores_fpn)

print("Mean IoU using Unet =", mean_iou_unet)
print("Mean IoU using Linknet =", mean_iou_linknet)
print("Mean IoU using FPN =", mean_iou_fpn)


In [ ]:
# Visualize results for a random test image
test_img_number = np.random.randint(len(X_test1))
test_img = X_test1[test_img_number]
ground_truth = y_test[test_img_number]

plt.figure(figsize=(16, 12))

# First row
plt.subplot(2, 3, 1)
plt.title('Testing Image')
plt.imshow(test_img[:, :, 0], cmap='gray')

plt.subplot(2, 3, 2)
plt.title('Ground Truth')
plt.imshow(ground_truth[:, :, 0], cmap='jet')

# Second row
plt.subplot(2, 3, 3)
plt.title('Unet Prediction')
plt.imshow(y_pred_unet_binary[test_img_number, :, :, 0], cmap='jet')

plt.subplot(2, 3, 4)
plt.title('Linknet Prediction')
plt.imshow(y_pred_linknet_binary[test_img_number, :, :, 0], cmap='jet')

# Third row
plt.subplot(2, 3, 5)
plt.title('FPN Prediction')
plt.imshow(y_pred_fpn_binary[test_img_number, :, :, 0], cmap='jet')

plt.show()


In [ ]:
#BEFORE getting the best weight>>

# Define weights for each model
weights = [0.3, 0.2, 0.5]  # Adjust the weights as needed

# Combine predictions with the weighted average
weighted_ensemble_prediction = (
    y_pred_unet * weights[0] +
    y_pred_linknet * weights[1] +
    y_pred_fpn * weights[2]
)

# Threshold the ensemble prediction to obtain binary results
ensemble_binary = (weighted_ensemble_prediction > threshold).astype(np.uint8)

# Calculate IoU for the ensemble prediction
iou_scores_ensemble = []

for i in range(len(X_test1)):
    iou_ensemble = jaccard_score(y_test[i, :, :, 0].flatten(), ensemble_binary[i, :, :, 0].flatten(), average=None)
    iou_scores_ensemble.append(iou_ensemble)

mean_iou_ensemble = np.mean(iou_scores_ensemble)

print("Mean IoU of the Ensembled Model =", mean_iou_ensemble)


In [ ]:

from itertools import product

# Define the weights for each model

# Finding the best weights for an ensemble model can be done through a
# search process. One common approach is to perform a grid search or a random
# search over a range of possible weight combinations. The idea is to evaluate
# the performance of the ensemble for different weight combinations and choose the
# one that gives the best results.
# For example, weights_combinations = list(product([0.2, 0.3, 0.4, 0.5, 0.6], repeat=3))
# represents a search space with weights ranging from 0.2 to 0.6 for each model.


weights_combinations = list(product(np.arange(0.1, 1.0, 0.1), repeat=3)) # Adjust the weights as needed # this one will take some time
# weights_combinations = list(product(np.arange(0.1, 0.7, 0.1), repeat=3))
#weights_combinations = list(product([0.3, 0.2, 0.5], repeat=3))


ensemble_iou_scores = []

# Iterate through different weight combinations
for weights in weights_combinations:
    # Combine predictions with the weighted average
    weighted_ensemble_prediction = (
        y_pred_unet * weights[0] +
        y_pred_linknet * weights[1] +
        y_pred_fpn * weights[2]
    )

    # Threshold the ensemble prediction to obtain binary results
    ensemble_binary = (weighted_ensemble_prediction > threshold).astype(np.uint8)

    # Calculate IoU for the ensemble prediction
    iou_scores_ensemble = []

    for i in range(len(X_test1)):
        iou_ensemble = jaccard_score(y_test[i, :, :, 0].flatten(), ensemble_binary[i, :, :, 0].flatten(), average=None)
        iou_scores_ensemble.append(iou_ensemble)

    mean_iou_ensemble = np.mean(iou_scores_ensemble)
    ensemble_iou_scores.append(mean_iou_ensemble)

# Find the best weight combination
best_weight_index = np.argmax(ensemble_iou_scores)
best_weights = weights_combinations[best_weight_index]
best_iou = ensemble_iou_scores[best_weight_index]

#print("Best Weights Index:", best_weight_index)
print("Best Weights:", best_weights)
print("Best Mean IoU:", best_iou)


# Visualize the results
plt.figure(figsize=(12, 8))
plt.plot(range(len(weights_combinations)), ensemble_iou_scores, marker='o', linestyle='-')
plt.title("Mean IoU for Different Weight Combinations")
plt.xlabel("Weight Combination Index")
plt.ylabel("Mean IoU")
plt.xticks(range(len(weights_combinations)), range(len(weights_combinations)))
plt.xticks(rotation=90)
plt.grid(True)

plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming weights_combinations contains the weight combinations
# and ensemble_iou_scores contains the corresponding IoU scores

# Convert weights_combinations to numpy array for easier manipulation
weights_combinations = np.array(weights_combinations)

# Plot the weights combinations and IoU scores
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for weight combinations and IoU scores
sc = ax.scatter(weights_combinations[:,0], weights_combinations[:,1], weights_combinations[:,2], c=ensemble_iou_scores, cmap='viridis')

# Set labels and title
ax.set_xlabel('UNet Weight')
ax.set_ylabel('LinkNet Weight')
ax.set_zlabel('FPN Weight')
ax.set_title('Weight Combinations and IoU Scores')

# Add color bar
cbar = fig.colorbar(sc)
cbar.set_label('IoU Score')

# Find the index of the best weights
best_weight_index = np.argmax(ensemble_iou_scores)
best_weights = weights_combinations[best_weight_index]
best_iou = ensemble_iou_scores[best_weight_index]

# Highlight the best weights
ax.scatter(best_weights[0], best_weights[1], best_weights[2], c='red', s=100, label='Best Weights')

# Annotate the best weights with their IoU score, offsetting the text slightly
# ax.text(best_weights[0], best_weights[1], best_weights[2], f'({best_weights[0]:.2f}, {best_weights[1]:.2f}, {best_weights[2]:.2f})\nIoU: {best_iou:.2f}', color='black', verticalalignment='bottom', horizontalalignment='center', zorder=20)
ax.text(best_weights[0], best_weights[1], best_weights[2], f'({best_weights[0]:.2f}, {best_weights[1]:.2f}, {best_weights[2]:.2f})\nIoU: {best_iou:.5f}', color='black', verticalalignment='bottom', horizontalalignment='center', zorder=20)

# Add legend
ax.legend()

plt.show()


In [ ]:
import pickle  # You can also use other serialization methods

# Save the best weights to a file
with open('best_weights.pkl', 'wb') as file:
    pickle.dump(best_weights, file)


In [ ]:
import pickle

# Load the best weights from the file
with open('best_weights.pkl', 'rb') as file:
    loaded_best_weights = pickle.load(file)


In [ ]:
#AFTER>>
#CHEACK>>

import pickle

# Load the best weights from the file
with open('best_weights.pkl', 'rb') as file:
    best_weights = pickle.load(file)


# Combine predictions with the weighted average
weighted_ensemble_prediction_best = (
    y_pred_unet * best_weights[0] +
    y_pred_linknet * best_weights[1] +
    y_pred_fpn * best_weights[2]
)

# Threshold the ensemble prediction to obtain binary results
ensemble_binary_best = (weighted_ensemble_prediction_best > threshold).astype(np.uint8)

# Calculate IoU for the ensemble prediction
iou_scores_ensemble = []

for i in range(len(X_test1)):
    iou_ensemble = jaccard_score(y_test[i, :, :, 0].flatten(), ensemble_binary_best[i, :, :, 0].flatten(), average=None)
    iou_scores_ensemble.append(iou_ensemble)

mean_iou_ensemble = np.mean(iou_scores_ensemble)

print("Mean IoU of the Ensembled Model =", mean_iou_ensemble)




In [ ]:
import random
import matplotlib.pyplot as plt

# Randomly select a test image
test_img_number = random.randint(0, len(X_test1) - 1)

test_img = X_test1[test_img_number]
ground_truth = y_test[test_img_number][:, :, 0]
ensemble_prediction = ensemble_binary_best[test_img_number][:, :, 0]
unet_prediction = y_pred_unet_binary[test_img_number][:, :, 0]
linknet_prediction = y_pred_linknet_binary[test_img_number][:, :, 0]
fpn_prediction = y_pred_fpn_binary[test_img_number][:, :, 0]

# Create subplots to display the images
plt.figure(figsize=(16, 12))

# Original test image
plt.subplot(2, 3, 1)
plt.title('Testing Image')
plt.imshow(test_img[:, :, 0], cmap='gray')

# Ground Truth
plt.subplot(2, 3, 2)
plt.title('Ground Truth')
plt.imshow(ground_truth, cmap='jet')

# Ensemble Prediction
plt.subplot(2, 3, 3)
plt.title('Ensemble Prediction')
plt.imshow(ensemble_prediction, cmap='jet')

# Unet Prediction
plt.subplot(2, 3, 4)
plt.title('Unet Prediction')
plt.imshow(unet_prediction, cmap='jet')

# Linknet Prediction
plt.subplot(2, 3, 5)
plt.title('Linknet Prediction')
plt.imshow(linknet_prediction, cmap='jet')

# FPN Prediction
plt.subplot(2, 3, 6)
plt.title('FPN Prediction')
plt.imshow(fpn_prediction, cmap='jet')

plt.show()


In [ ]:
from sklearn.metrics import precision_score, accuracy_score

# Assuming ground truth labels are binary (0 or 1)
ground_truth_binary = y_test[:, :, :, 0].flatten()

# Flatten the predictions of the ensemble model
ensemble_predictions_binary = ensemble_binary_best[:, :, :, 0].flatten()

# Calculate precision and accuracy
precision_ensemble = precision_score(ground_truth_binary, ensemble_predictions_binary)
accuracy_ensemble = accuracy_score(ground_truth_binary, ensemble_predictions_binary)

print("Precision of the Ensemble Model:", precision_ensemble)
print("Accuracy of the Ensemble Model:", accuracy_ensemble)


In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from keras.models import load_model
from keras.models import Model
from keras.layers import Input
from keras import backend as K
import segmentation_models as sm

# Load individual models (unet, linknet, fpn)
model_unet = load_model('/content/unet_res34_backbone_8epochs.hdf5', compile=False)
model_linknet = load_model('/content/linknet_res34_backbone_8epochs.hdf5', compile=False)
model_fpn = load_model('/content/FPN_res34_backbone_8epochs.hdf5', compile=False)

# Assuming you have defined the weights and threshold
threshold = 0.5

# Load the best weights from the file
with open('best_weights.pkl', 'rb') as file:
    best_weights = pickle.load(file)

# Create a shared input tensor with the desired shape
shared_input_tensor = Input(shape=(256, 256, 3))

# Pass the shared input tensor to each model
output_unet = model_unet(shared_input_tensor)
output_linknet = model_linknet(shared_input_tensor)
output_fpn = model_fpn(shared_input_tensor)

# Combine predictions with the loaded best weights
weighted_ensemble_prediction = (
    output_unet * best_weights[0] +
    output_linknet * best_weights[1] +
    output_fpn * best_weights[2]
)

# Threshold the ensemble prediction to obtain binary results
ensemble_binary = K.cast(weighted_ensemble_prediction > threshold, dtype=K.floatx())

# Create a model with the same input tensor and the ensemble output
ensemble_model = Model(inputs=shared_input_tensor, outputs=ensemble_binary)

# Save the ensemble model
ensemble_model.save('/content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/ensemble_model_with_weights_3.hdf5')


In [ ]:
from keras.models import load_model

# Load the ensemble model
ensemble_model = load_model('/content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/ensemble_model_with_weights_3.hdf5', compile=False)

from keras.preprocessing import image
import numpy as np

# Load the image from drive
img_path = '/content/drive/MyDrive/Crack/IMG_20240110_072919.jpg'  # Update with your image path
img = image.load_img(img_path, target_size=(256, 256))  # Adjust target size if needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
preprocessed_image = preprocess_input1(img_array)  # Assuming you have a preprocessing function


predictions = ensemble_model.predict(preprocessed_image)


threshold = 0.5
binary_mask = (predictions > threshold).astype(np.uint8)


import matplotlib.pyplot as plt

# Display the original image
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title('Original Image')

# Display the segmentation result
plt.subplot(1, 2, 2)
plt.imshow(binary_mask[0, :, :, 0], cmap='gray')  # Assuming a single-channel mask
plt.title('final ensembled Result')

plt.show()





In [ ]:
from keras.models import load_model

# Load the ensemble model
ensemble_model = load_model('/content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/ensemble_model_with_weights_3.hdf5', compile=False)

from keras.preprocessing import image
import numpy as np

# Load the image from drive
img_path = '/content/drive/MyDrive/Crack/IMG_20240110_072919.jpg'
img = image.load_img(img_path, target_size=(256, 256))  # Adjust target size if needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
preprocessed_image = preprocess_input1(img_array)  # Assuming you have a preprocessing function

predictions = ensemble_model.predict(preprocessed_image)

threshold = 0.5
binary_mask = (predictions > threshold).astype(np.uint8)

import matplotlib.pyplot as plt

# Display the original image
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title('Original Image')

# Display the segmentation result
plt.subplot(1, 2, 2)
plt.imshow(binary_mask[0, :, :, 0], cmap='gray')  # Assuming a single-channel mask
plt.title('Final ensembled Result')

plt.show()

# Calculate crack area (you'll need to replace this with your own function)
def calculate_area(binary_mask):
    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize area variable
    area = 0

    # Calculate area for each contour
    for contour in contours:
        area += cv2.contourArea(contour)

    return area

# Now, replace the placeholder function call with the actual area calculation
crack_area = calculate_area(binary_mask[0, :, :, 0])

# thresholds for area to differentiate between major and minor cracks
major_threshold = 900
minor_threshold = 600

# Determine if the crack is major or minor based on its area
if crack_area > major_threshold:
    print("This crack is classified as a major crack.")
elif crack_area > minor_threshold:
    print("This crack is classified as a minor crack.")
else:
    print("This crack is classified as not significant.")


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

# Setting the root directory
class ROOTDIR:
    train_images_dir = "../content/drive/MyDrive/ColabNotebooks/Data/504_ALLCRACKED_DATASET/images/"

# Get all image paths in the specified directory
def get_image_paths(root_dir):
    image_paths = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # mostly jpg
                image_paths.append(os.path.join(root, file))
    return image_paths

# Get all image paths
all_image_paths = get_image_paths(ROOTDIR.train_images_dir)

# number of images to display (10 )
num_images_to_display = min(10, len(all_image_paths))

# Randomly select num_images_to_display image paths from the dataset
random_image_paths = random.sample(all_image_paths, num_images_to_display)

# figure for 2 rows and 5 columns
fig, axes = plt.subplots(2, 5, figsize=(15, 8))

for i, img_path in enumerate(random_image_paths):
    # Loading/processing img
    img = Image.open(img_path)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    preprocessed_image = preprocess_input1(img_array)

    predictions = ensemble_model.predict(preprocessed_image)

    threshold = 0.5
    binary_mask = (predictions > threshold).astype(np.uint8)



    # Calculate crack area (you'll need to replace this with your own function)
    crack_area = calculate_area(binary_mask[0, :, :, 0])

    # thresholds for area to differentiate between major and minor cracks
    major_threshold = 900
    minor_threshold = 600

    # checking if the crack is major or minor based on its area
    if crack_area > major_threshold:
        classification = "Major Crack"
    elif crack_area > minor_threshold:
        classification = "Minor Crack"
    else:
        classification = "Not Significant"

    # Ploting images
    row = i // 5
    col = i % 5
    axes[row, col].imshow(img)
    axes[row, col].set_title(f"Classification: {classification}")
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
from keras.preprocessing import image  # Import the missing module

# Load the ensemble model and preprocess_input1 function
# Load the model and define the preprocess_input1 function
# ... (code for loading model and preprocess function)

# Setting the root directory for images
class ROOTDIR:
    train_images_dir = "/content/drive/MyDrive/Crack"  # Update with your directory

# Function to get image paths in a specified directory
def get_image_paths(root_dir):
    image_paths = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Mostly jpg
                image_paths.append(os.path.join(root, file))
    return image_paths

# Get all image paths
all_image_paths = get_image_paths(ROOTDIR.train_images_dir)

# Select 5 random image paths
num_images_to_display = min(5, len(all_image_paths))
random_image_paths = random.sample(all_image_paths, num_images_to_display)

# Manually set the figure size and spacing between subplots
fig_width = 14
fig_height = 12
fig, axes = plt.subplots(3, 5, figsize=(fig_width, fig_height))

# Adjust spacing between subplots manually
fig.subplots_adjust(top=0.88, bottom=0.1, left=0.02, right=0.98, hspace=0.01, wspace=0.1)

for i, img_path in enumerate(random_image_paths):
    # Load the image
    img = Image.open(img_path)
    target_size = (256, 256)  # Adjust target size if needed
    img = img.resize(target_size)

    # Preprocess the image for prediction
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    preprocessed_image = preprocess_input1(img_array)  # Modify as needed

    # Get predictions from the ensemble model
    predictions = ensemble_model.predict(preprocessed_image)

    threshold = 0.5
    binary_mask = (predictions > threshold).astype(np.uint8)

    # Calculate crack area using contours (Replace this with your function)
    crack_area = calculate_area(binary_mask[0, :, :, 0])

    # Set thresholds for classification
    major_threshold = 900
    minor_threshold = 600

    # Determine crack classification based on area
    if crack_area > major_threshold:
        classification = "Major Crack"
    elif crack_area > minor_threshold:
        classification = "Minor Crack"
    else:
        classification = "Not Significant"

    # Display original images
    axes[0, i].imshow(img)
    axes[0, i].set_title(f"Original Image")
    axes[0, i].axis('off')

    # Display images with ensemble segmentation
    axes[1, i].imshow(img)
    axes[1, i].set_title(f"Final Ensemble Segmentation")  # Show ensemble segmentation in the title
    axes[1, i].axis('off')
    axes[1, i].imshow(binary_mask[0, :, :, 0], cmap='hot', alpha=0.5)  # Overlay crack segmentation result in yellow
    axes[1, i].set_aspect('equal')  # Set aspect ratio to equal

    # Display images with classification
    axes[2, i].imshow(img)
    axes[2, i].set_title(f"Classification: {classification}")  # Show classification in the title
    axes[2, i].axis('off')
    crack_image = binary_mask[0, :, :, 0]
    crack_image_colored = np.zeros((*crack_image.shape, 3), dtype=np.uint8)  # Create RGB image
    crack_image_colored[crack_image == 1] = [255, 0, 0]  # Set cracks to red color (Modify as needed)
    axes[2, i].imshow(crack_image_colored, alpha=0.3)  # Overlay crack segmentation result

plt.show()


In [ ]:
!git clone https://github.com/oskaro01/concrete_crack_detection-we-dl-ml-cv.git


In [ ]:
!ls /content/


In [ ]:
!ls /content/concrete_crack_detection-we-dl-ml-cv/


In [ ]:
!git clone https://github.com/oskaro01/concrete_crack_detection-we-dl-ml-cv.git


In [ ]:
!mv /content/concrete_crack_detection-we-dl-ml-cv/upload.ipynb /content/concrete_crack_detection-we-dl-ml-cv/


In [30]:
!ls /content/



concrete_crack_detection-we-dl-ml-cv  drive  sample_data


In [31]:
!find /content/ -type f -name "*.ipynb"



/content/drive/MyDrive/ColabNotebooks/Untitled0.ipynb
/content/drive/MyDrive/ColabNotebooks/U-net.ipynb
/content/drive/MyDrive/ColabNotebooks/unet attmpt2.ipynb
/content/drive/MyDrive/ColabNotebooks/Unet-attmpt3suc.ipynb
/content/drive/MyDrive/ColabNotebooks/kaggle to colab dataset.ipynb
/content/drive/MyDrive/Colab Notebooks/code from datasetSUC.ipynb
/content/drive/MyDrive/Colab Notebooks/code_from_datasetSUC (1) (3).ipynb
/content/drive/MyDrive/Colab Notebooks/code_from_datasetSUC (1) (2).ipynb
/content/drive/MyDrive/Colab Notebooks/code_from_datasetSUC (1) (1).ipynb
/content/drive/MyDrive/Colab Notebooks/code_from_datasetSUC (1).ipynb
/content/drive/MyDrive/Colab Notebooks/ENSEM2.ipynb
/content/drive/MyDrive/Colab Notebooks/unet and linknet attempt3.ipynb
/content/drive/MyDrive/Colab Notebooks/ensemble.ipynb
/content/drive/MyDrive/Colab Notebooks/m10unetlinknetfpn.ipynb
/content/drive/MyDrive/Colab Notebooks/f50Untitled0.ipynb
/content/drive/MyDrive/Colab Notebooks/SGDtesting.ipynb

In [29]:
from google.colab import files
files.download('concretecrack.ipynb')


FileNotFoundError: Cannot find file: concretecrack.ipynb